In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import zscore,iqr
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.metrics import classification_report,confusion_matrix, mean_absolute_error,mean_squared_error
from IPython.display import Audio
from sklearn.linear_model import LinearRegression,LogisticRegression
from imblearn.combine import SMOTETomek

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers, optimizers

In [83]:
df = pd.read_csv('train_v9rqX0R.csv')
df1 = df.copy()
sound_file = 'C:\\Users\\raghavendra.s.k\\Music\\Notification.mp3'
def notify():
    return Audio(sound_file, autoplay=True)

In [84]:
# step1
def step1(df1):
    df1_na = df1.dropna(axis=0)
    df1_nona = pd.DataFrame(df1_na.groupby('Item_Identifier')['Item_Weight'].mean()).reset_index()
    II_W1 = pd.DataFrame(df1.groupby('Item_Type')['Item_Weight'].median().sort_values(ascending=False),columns=['Item_Weight']).reset_index()    
    def weight_update(x,y):
        try:
            if pd.isna(x):
                return df1_nona[(df1_nona['Item_Identifier']==y)]['Item_Weight'].values[0]
            else:
                return x
        except:
            return x
    def fill_itemweight2(x,z):
        try:
            if pd.isna(z):
                return II_W1[(II_W1['Item_Type']==x)]['Item_Weight'].values[0]
            else:
                return z
        except:
            return z
    df1['Item_Weight']=df1.apply(lambda x: weight_update(x['Item_Weight'],x['Item_Identifier']),axis=1)
    df1['Item_Weight']=df1.apply(lambda x: fill_itemweight2(x['Item_Type'],x['Item_Weight']),axis=1)
    def item_fat_content(x):
        if x == 'low fat' or x == 'LF':
            return 'Low Fat'
        elif x== 'reg':
            return 'Regular'
        else:
            return x
    df1['Item_Fat_Content'] = df1['Item_Fat_Content'].apply(item_fat_content)
    df1 = pd.concat([df1,pd.get_dummies(df1['Item_Fat_Content'],drop_first=True)],axis=1)
    II_V1 = pd.DataFrame(df1.groupby(['Item_Identifier','Item_Fat_Content'])['Item_Visibility'].median().reset_index())
    def visibility_update(x,y,z):
        if z == 0:
            return II_V1[(II_V1['Item_Identifier'] == x) & (II_V1['Item_Fat_Content']==y )]['Item_Visibility'].values[0]
        else:
            return z
    df1['Item_Visibility'] = df1.apply(lambda x: visibility_update(x['Item_Identifier'],x['Item_Fat_Content'],x['Item_Visibility']),axis=1)
        
    df1.drop('Item_Fat_Content',axis=1,inplace=True)
    def year(x):
        if x == 1987 or x == 2004:
            return 6
        elif x == 1999 or x == 2007:
            return 5
        elif x == 1985 or x == 1997:
            return 4
        elif x==2002:
            return 3
        elif x == 2009:
            return 2
        elif x == 1998:
            return 1
        else:
            return 1   
    df1['Outlet_Establishment_Year'] = df1['Outlet_Establishment_Year'].apply(year)    
    df1['Outlet_Type'].replace({'Grocery Store':1,'Supermarket Type2':2,'Supermarket Type1':3,'Supermarket Type3':4},inplace=True)
    df1['Item_Type'] = df1['Item_Type'].replace({'Seafood':6,'Household':5,'Starchy Foods':5,'Snack Foods':5,'Breads':4,'Canned':4
                                 ,'Fruits and Vegetables':4,'Meat':4,'Hard Drinks':4,'Others':3,'Frozen Foods':2,
                                  'Health and Hygiene':2,'Dairy':2,'Baking Goods':1,'Breakfast':1,'Soft Drinks':1})
    df1['Outlet_Location_Type'] = df1['Outlet_Location_Type'].replace({'Tier 1':1,'Tier 2':3,'Tier 3':2})
    df1.drop('Item_Identifier',axis=1,inplace=True)
    df1['Outlet_Identifier'].replace({'OUT027':4,'OUT035':3,'OUT013':3,'OUT017':3,'OUT049':2,'OUT046':2,'OUT045':2,'OUT018':2,'OUT019':1,'OUT010':1},inplace=True)
    df1['Regular'] = df1['Regular'].apply(int)
    return df1

In [85]:
def outlet_size(df):
    df.drop('Item_Outlet_Sales',axis=1,inplace=True)
    df1_na = df.dropna(axis=0)
    RFC = RandomForestClassifier()
    X = df1_na.drop(['Outlet_Size'],axis=1)
    y = df1_na['Outlet_Size']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    RFC.fit(X_train,y_train)
    pred = RFC.predict(X_test)
    print(classification_report(y_test,pred))
    pred_df = pd.DataFrame(pred,columns=['predicted'])
    pred_df['Actual'] = y_test.values
    def check(x,y):
        if x == y:
            return True
        else:
            return False
    pred_df['Check'] = pred_df.apply(lambda x: check(x['predicted'],x['Actual']),axis=1)   
    df['Outlet_Size1']=RFC.predict(df.drop('Outlet_Size',axis=1))
    df['Check'] = df.apply(lambda x: check(x['Outlet_Size'],x['Outlet_Size1']),axis=1)
    df.drop(['Outlet_Size','Check'],axis=1,inplace=True)
    df.columns = ['Item_Weight', 'Item_Visibility', 'Item_Type', 'Item_MRP',
           'Outlet_Identifier', 'Outlet_Establishment_Year',
           'Outlet_Location_Type', 'Outlet_Type','Regular',
           'Outlet_Size']
    df['Outlet_Size'] = df['Outlet_Size'].replace({'Medium':3,'High':2,'Small':1})   
    return df
    

In [86]:
def outlet_size_test(df):
    df1_na = df.dropna(axis=0)
    RFC = RandomForestClassifier()
    X = df1_na.drop(['Outlet_Size'],axis=1)
    y = df1_na['Outlet_Size']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    RFC.fit(X_train,y_train)
    pred = RFC.predict(X_test)
    print(classification_report(y_test,pred))
    pred_df = pd.DataFrame(pred,columns=['predicted'])
    pred_df['Actual'] = y_test.values
    def check(x,y):
        if x == y:
            return True
        else:
            return False
    pred_df['Check'] = pred_df.apply(lambda x: check(x['predicted'],x['Actual']),axis=1)   
    df['Outlet_Size1']=RFC.predict(df.drop('Outlet_Size',axis=1))
    df['Check'] = df.apply(lambda x: check(x['Outlet_Size'],x['Outlet_Size1']),axis=1)
    df.drop(['Outlet_Size','Check'],axis=1,inplace=True)
    df.columns = ['Item_Weight', 'Item_Visibility', 'Item_Type', 'Item_MRP',
           'Outlet_Identifier', 'Outlet_Establishment_Year',
           'Outlet_Location_Type', 'Outlet_Type','Regular',
           'Outlet_Size']
    df['Outlet_Size'] = df['Outlet_Size'].replace({'Medium':3,'High':2,'Small':1})   
    df['Regular'] = df['Regular'].apply(int)
    return df
    

In [87]:
test = pd.read_csv('test_AbJTz2l.csv')
test_1 = step1(test)

In [88]:
test_1_o = outlet_size_test(test_1)

              precision    recall  f1-score   support

        High       1.00      1.00      1.00       187
      Medium       1.00      1.00      1.00       572
       Small       1.00      1.00      1.00       464

    accuracy                           1.00      1223
   macro avg       1.00      1.00      1.00      1223
weighted avg       1.00      1.00      1.00      1223



# Training data

In [89]:
df1 = step1(df1)
df1_filled = outlet_size(df1)
df1_filled = pd.concat([df1_filled,df['Item_Outlet_Sales']],axis=1)

              precision    recall  f1-score   support

        High       1.00      1.00      1.00       288
      Medium       1.00      1.00      1.00       826
       Small       1.00      1.00      1.00       720

    accuracy                           1.00      1834
   macro avg       1.00      1.00      1.00      1834
weighted avg       1.00      1.00      1.00      1834



In [90]:

X = df1_filled.drop(['Item_Outlet_Sales','Item_Weight','Regular','Item_Type'],axis=1)
y = df1_filled['Item_Outlet_Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=11)

In [13]:
#best model - 387th rank

# RFR = RandomForestRegressor(n_estimators=700,min_impurity_decrease= 0.60,n_jobs=6 ,random_state =11,max_samples=306, max_depth=6,criterion='mse',min_samples_split=9,min_samples_leaf=2,warm_start=True,max_features=5,ccp_alpha=0.45,max_leaf_nodes=45,oob_score=True)
# RFR.fit(X_train,y_train)
# pred = RFR.predict(X_test)
# mean_absolute_error(y_test,pred)

In [14]:
#Best model - #329
# RFR = RandomForestRegressor(n_estimators=800,min_impurity_decrease= 0.60,n_jobs=7 ,random_state =11,max_samples=306, max_depth=6,criterion='mse',min_samples_split=9,min_samples_leaf=2,warm_start=True,max_features=5,ccp_alpha=0.45,max_leaf_nodes=47,oob_score=True)
# RFR.fit(X,y)
# pred = RFR.predict(X_test)
# mean_absolute_error(y_test,pred)

In [15]:
#313
# RFR = RandomForestRegressor(n_estimators=800,min_impurity_decrease= 0.60,n_jobs=7 ,random_state = 11,max_samples=306, max_depth=7,criterion='mse',min_samples_split=9,min_samples_leaf=2,warm_start=True,max_features=5,ccp_alpha=0.45,max_leaf_nodes=47,oob_score=True)
# RFR.fit(X_train,y_train)
# pred = RFR.predict(X_test)
# mean_absolute_error(y_test,pred)

#With full data #274

In [16]:
# RFR = RandomForestRegressor(n_estimators=800,min_impurity_decrease= 0.3,n_jobs=30,random_state = 11,max_samples=306, max_depth=7,criterion='mse' ,min_samples_split=9,min_samples_leaf=2,warm_start=True,max_features=5,ccp_alpha=0.78,max_leaf_nodes=47,oob_score=True)
# RFR.fit(X,y)

#With full 2#67

In [17]:
# RFR = RandomForestRegressor(n_estimators=800,min_impurity_decrease= 0.3,n_jobs=30,random_state = 11,max_samples=306, max_depth=7,criterion='mse' ,min_samples_split=9,min_samples_leaf=2,warm_start=True,max_features=5,ccp_alpha=0.78,max_leaf_nodes=47,oob_score=True)
# RFR.fit(X,y)
# pred = RFR.predict(X_test)
# mean_absolute_error(y_test,pred)

# Rank #235, filled the itemvisibility 0s to a value 

In [46]:
RFR = RandomForestRegressor(n_estimators=810,min_impurity_decrease= 0.3,n_jobs=30,random_state = 11,
                            max_samples=300, max_depth=7,criterion='mse' ,min_samples_split=9,
                            min_samples_leaf=2,warm_start=True,max_features=5,ccp_alpha=0.78,
                            max_leaf_nodes=49,oob_score=True)
RFR.fit(X_train,y_train)
pred = RFR.predict(X_test)
mean_absolute_error(y_test,pred)

RandomForestRegressor(ccp_alpha=0.78, max_depth=7, max_features=5,
                      max_leaf_nodes=49, max_samples=300,
                      min_impurity_decrease=0.3, min_samples_leaf=2,
                      min_samples_split=9, n_estimators=810, n_jobs=30,
                      oob_score=True, random_state=11, warm_start=True)

In [47]:
np.sqrt(mean_squared_error(y_test,pred))

1085.0347092203463

In [48]:
# notify()

#758.303286
#1085.114490

In [49]:
prediction51 = pd.DataFrame(test[['Item_Identifier','Outlet_Identifier']])
# test_1_o = test_1_o.drop(['Item_Weight','Regular','Item_Type'],axis=1)
prediction51['Item_Outlet_Sales'] = RFR.predict(test_1_o)
prediction51.to_csv('submission51.csv',index=False)

# Gradient boost classfier

In [50]:
# gra = GradientBoostingRegressor(n_estimators=70,learning_rate=0.1,criterion='mse',min_samples_split=63
#                                 ,min_samples_leaf=65,min_impurity_decrease=0.1,min_impurity_split=0.2,random_state=30)
# GSCV_gra = GridSearchCV(estimator=gra,param_grid={'n_estimators':[50,80,100,120,150,180,200,300]})
# gra.fit(X,y)


#Fetched 165th rank with 1145 RMSE value - The best till now and better than RFR

In [61]:
# gra = GradientBoostingRegressor(n_estimators=78,learning_rate=0.1,criterion='mse',min_samples_split=63
#                                 ,min_samples_leaf=65,min_impurity_decrease=0.1,min_impurity_split=0.2,
#                                    random_state=30)
# GSCV_gra = GridSearchCV(estimator=gra,param_grid={'n_estimators':[50,80,100,120,150,180,200,300]})
# gra.fit(X,y)

#Fetched 165th rank with 1134 RMSE value - The best till now and better than RFR

In [66]:
gra = GradientBoostingRegressor(n_estimators=78,learning_rate=0.1,criterion='mse',min_samples_split=63
                                ,min_samples_leaf=65,min_impurity_decrease=0.1,min_impurity_split=0.2,
                                   random_state=30,max_depth=4)

#     min_weight_fraction_leaf=0.0,
#     max_depth=3,
#     min_impurity_decrease=0.0,
#     min_impurity_split=None,
#     init=None,
#     random_state=None,
#     max_features=None,
#     alpha=0.9,
#     verbose=0,
#     max_leaf_nodes=None,
#     warm_start=False,
#     validation_fraction=0.1,
#     n_iter_no_change=None,
#     tol=0.0001,
#     ccp_alpha=0.0,




GSCV_gra = GridSearchCV(estimator=gra,param_grid={'n_estimators':[50,80,100,120,150,180,200,300]})
gra.fit(X,y)
# pred1 = gra.predict(X_test)
# mean_absolute_error(y_test,pred1)

#Not good - try to tune on all parameters

GradientBoostingRegressor(criterion='mse', max_depth=4,
                          min_impurity_decrease=0.1, min_impurity_split=0.2,
                          min_samples_leaf=65, min_samples_split=63,
                          n_estimators=78, random_state=30)

In [67]:
# np.sqrt(mean_squared_error(y_test,pred1))

#758.2624801759463
#1085.7502188821288

In [68]:
prediction52 = pd.DataFrame(test[['Item_Identifier','Outlet_Identifier']])
# test_1_o = test_1_o.drop(['Item_Weight','Regular','Item_Type'],axis=1)
prediction52['Item_Outlet_Sales'] = gra.predict(test_1_o)
prediction52.to_csv('submission52.csv',index=False)

In [69]:
prediction52[prediction52['Item_Outlet_Sales']<0]

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales


In [27]:
# notify()

# XGboost 

In [203]:
import xgboost as xgb
# xg_reg = xgb.XGBRegressor(objective ='reg:tweedie',eta=0.1,min_child_weight = 3,
#                           subsample = 0.9,colsample_bytree = 1, nthread = 3,eval_metric = "rmse",
#                 max_depth = 5, learning_rate = 0.1, alpha = 10, n_estimators = 100)

# colsample_bytree = 0.3, learning_rate = 0.1,, alpha = 10, n_estimators = 100

xg_reg = xgb.XGBRegressor()
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
mean_absolute_error(y_test,preds)



807.0991751997865

In [204]:
np.sqrt(mean_squared_error(y_test,preds))

1159.4096104546718

In [205]:
prediction23 = pd.DataFrame(test[['Item_Identifier','Outlet_Identifier']])
prediction23['Item_Outlet_Sales'] = xg_reg.predict(test_1_o)
prediction23.to_csv('submission23.csv',index=False)

In [206]:
prediction23[prediction52['Item_Outlet_Sales']<0]

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales


In [112]:
df1_filled['Item_MRP']

,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Regular,Outlet_Size,Item_Outlet_Sales
0,9.300,0.016047,2,249.8092,2,5,1,3,0,3,3735.1380
1,5.920,0.019278,1,48.2692,2,2,2,2,1,3,443.4228
2,17.500,0.016760,4,141.6180,2,5,1,3,0,3,2097.2700
3,19.200,0.107479,4,182.0950,1,1,2,1,1,3,732.3800
4,8.930,0.064197,5,53.8614,3,6,2,3,0,2,994.7052
...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,0.056783,5,214.5218,3,6,2,3,0,2,2778.3834
8519,8.380,0.046982,1,108.1570,2,3,3,3,1,3,549.2850
8520,10.600,0.035186,2,85.1224,3,6,3,3,0,1,1193.1136
8521,7.210,0.145221,5,103.1332,2,2,2,2,1,3,1845.5976


In [82]:
df1_filled['Item_MRP'].describe()

count    8523.000000
mean      140.992782
std        62.275067
min        31.290000
25%        93.826500
50%       143.012800
75%       185.643700
max       266.888400
Name: Item_MRP, dtype: float64

In [84]:
df1_filled['Item_Weight'].describe()

count    8523.000000
mean       12.879887
std         4.640188
min         4.555000
25%         8.792500
50%        12.650000
75%        16.850000
max        21.350000
Name: Item_Weight, dtype: float64

In [94]:
df1_filled.iloc[5]

Item_Weight                   10.395000
Item_Visibility                0.064471
Item_Type                      1.000000
Item_MRP                      51.400800
Outlet_Identifier              2.000000
Outlet_Establishment_Year      2.000000
Outlet_Location_Type           2.000000
Outlet_Type                    2.000000
Regular                        1.000000
Outlet_Size                    3.000000
Item_Outlet_Sales            556.608800
Name: 5, dtype: float64

In [100]:
II_V1 = pd.DataFrame(df.groupby(['Item_Identifier','Item_Fat_Content'])['Item_Visibility'].median().sort_values(ascending=False),columns=['Item_Weight']).reset_index()    
# def weight_update(x,y):
#     try:
#         if pd.isna(x):
#             return df1_nona[(df1_nona['Item_Identifier']==y)]['Item_Weight'].values[0]
#         else:
#             return x
#     except:
#         return x

In [115]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'low fat':'Low Fat','LF':'Low Fat','reg':'Regular'})

In [116]:
df['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular'], dtype=object)

In [122]:
II_V1 = pd.DataFrame(df.groupby(['Item_Identifier','Item_Fat_Content'])['Item_Visibility'].median().reset_index())

In [128]:
def visibility_update(x,y,z):
    if z == 0:
        return II_V1[(II_V1['Item_Identifier'] == x) & (II_V1['Item_Fat_Content']==y )]['Item_Visibility'].values[0]
    else:
        return z
        

In [130]:
df.apply(lambda x: visibility_update(x['Item_Identifier'],x['Item_Fat_Content'],x['Item_Visibility']),axis=1)

0       0.016047
1       0.019278
2       0.016760
3       0.022861
4       0.006590
          ...   
8518    0.056783
8519    0.046982
8520    0.035186
8521    0.145221
8522    0.044878
Length: 8523, dtype: float64

# Neural Network Regressor

In [92]:
from sklearn.preprocessing import StandardScaler

In [121]:
SS1 = StandardScaler()

In [95]:
X_trains = SS1.fit_transform(X_train)
X_tests = SS1.fit_transform(X_test)

In [122]:
Xs = SS1.fit_transform(X)

In [180]:
ES = EarlyStopping(monitor='mean_squared_error',mode='min',verbose=1,patience=21)
model6 = Sequential()
model6.add(Dense(50,input_shape=(7,),activation='relu'))
model6.add(Dense(40,activation='relu'))
model6.add(Dense(30,activation='relu'))
model6.add(Dense(20,activation='relu'))
model6.add(Dense(10,activation='relu'))
model6.add(Dense(5,activation='relu'))
model6.add(Dense(1,activation='relu'))
adam = optimizers.Adam(lr=0.05, decay=1e-3)
model6.compile(optimizer=adam,metrics='mean_squared_error',loss='mean_squared_error')
model6.fit(X_trains,y_train,epochs=500,validation_data=(X_tests,y_test),callbacks=ES)

Epoch 1/500
187/187 [==============================] - 2s 3ms/step - loss: 1632528.7500 - mean_squared_error: 1632528.7500 - val_loss: 1368948.7500 - val_mean_squared_error: 1368948.7500
Epoch 2/500
187/187 [==============================] - 0s 2ms/step - loss: 1394057.6250 - mean_squared_error: 1394057.6250 - val_loss: 1486940.8750 - val_mean_squared_error: 1486940.8750
Epoch 3/500
187/187 [==============================] - 0s 2ms/step - loss: 1325776.7500 - mean_squared_error: 1325776.7500 - val_loss: 1204719.1250 - val_mean_squared_error: 1204719.1250
Epoch 4/500
187/187 [==============================] - 0s 2ms/step - loss: 1212297.3750 - mean_squared_error: 1212297.3750 - val_loss: 1219854.3750 - val_mean_squared_error: 1219854.3750
Epoch 5/500
187/187 [==============================] - 0s 2ms/step - loss: 1235513.3750 - mean_squared_error: 1235513.3750 - val_loss: 1183252.8750 - val_mean_squared_error: 1183252.8750
Epoch 6/500
187/187 [==============================] - 0s 2ms/ste

Epoch 88/500
187/187 [==============================] - 0s 2ms/step - loss: 1127171.3750 - mean_squared_error: 1127171.3750 - val_loss: 1176563.8750 - val_mean_squared_error: 1176563.8750
Epoch 89/500
187/187 [==============================] - 0s 2ms/step - loss: 1132039.3750 - mean_squared_error: 1132039.3750 - val_loss: 1172087.6250 - val_mean_squared_error: 1172087.6250
Epoch 90/500
187/187 [==============================] - 1s 3ms/step - loss: 1127748.3750 - mean_squared_error: 1127748.3750 - val_loss: 1179611.0000 - val_mean_squared_error: 1179611.0000
Epoch 91/500
187/187 [==============================] - 0s 3ms/step - loss: 1128904.0000 - mean_squared_error: 1128904.0000 - val_loss: 1178909.8750 - val_mean_squared_error: 1178909.8750
Epoch 92/500
187/187 [==============================] - 1s 3ms/step - loss: 1125013.3750 - mean_squared_error: 1125013.3750 - val_loss: 1174905.6250 - val_mean_squared_error: 1174905.6250
Epoch 93/500
187/187 [==============================] - 0s 2

187/187 [==============================] - 0s 2ms/step - loss: 1113127.2500 - mean_squared_error: 1113127.2500 - val_loss: 1194597.3750 - val_mean_squared_error: 1194597.3750
Epoch 175/500
187/187 [==============================] - 0s 2ms/step - loss: 1113893.3750 - mean_squared_error: 1113893.3750 - val_loss: 1183903.1250 - val_mean_squared_error: 1183903.1250
Epoch 176/500
187/187 [==============================] - 0s 2ms/step - loss: 1114141.2500 - mean_squared_error: 1114141.2500 - val_loss: 1187562.1250 - val_mean_squared_error: 1187562.1250
Epoch 177/500
187/187 [==============================] - 0s 2ms/step - loss: 1111386.2500 - mean_squared_error: 1111386.2500 - val_loss: 1183652.7500 - val_mean_squared_error: 1183652.7500
Epoch 178/500
187/187 [==============================] - 0s 2ms/step - loss: 1107230.1250 - mean_squared_error: 1107230.1250 - val_loss: 1207676.7500 - val_mean_squared_error: 1207676.7500
Epoch 179/500
187/187 [==============================] - 0s 2ms/step 

187/187 [==============================] - 1s 3ms/step - loss: 1103567.1250 - mean_squared_error: 1103567.1250 - val_loss: 1187286.1250 - val_mean_squared_error: 1187286.1250
Epoch 261/500
187/187 [==============================] - 0s 3ms/step - loss: 1102903.8750 - mean_squared_error: 1102903.8750 - val_loss: 1187743.5000 - val_mean_squared_error: 1187743.5000
Epoch 262/500
187/187 [==============================] - 1s 3ms/step - loss: 1100996.2500 - mean_squared_error: 1100996.2500 - val_loss: 1187543.5000 - val_mean_squared_error: 1187543.5000
Epoch 263/500
187/187 [==============================] - 0s 2ms/step - loss: 1103385.6250 - mean_squared_error: 1103385.6250 - val_loss: 1186421.7500 - val_mean_squared_error: 1186421.7500
Epoch 264/500
187/187 [==============================] - 1s 3ms/step - loss: 1104096.6250 - mean_squared_error: 1104096.6250 - val_loss: 1189102.7500 - val_mean_squared_error: 1189102.7500
Epoch 265/500
187/187 [==============================] - 1s 3ms/step 

187/187 [==============================] - 0s 2ms/step - loss: 1098346.3750 - mean_squared_error: 1098346.3750 - val_loss: 1192066.0000 - val_mean_squared_error: 1192066.0000
Epoch 347/500
187/187 [==============================] - 0s 2ms/step - loss: 1096209.8750 - mean_squared_error: 1096209.8750 - val_loss: 1197341.8750 - val_mean_squared_error: 1197341.8750
Epoch 348/500
187/187 [==============================] - 0s 2ms/step - loss: 1096606.3750 - mean_squared_error: 1096606.3750 - val_loss: 1196872.6250 - val_mean_squared_error: 1196872.6250
Epoch 349/500
187/187 [==============================] - 0s 2ms/step - loss: 1098144.3750 - mean_squared_error: 1098144.3750 - val_loss: 1190871.2500 - val_mean_squared_error: 1190871.2500
Epoch 350/500
187/187 [==============================] - 0s 2ms/step - loss: 1098277.3750 - mean_squared_error: 1098277.3750 - val_loss: 1190414.8750 - val_mean_squared_error: 1190414.8750
Epoch 351/500
187/187 [==============================] - 0s 2ms/step 

In [181]:
notify()

In [182]:
np.sqrt(1131475.6250)

1063.7084304451103

In [183]:
prediction61 = pd.DataFrame(test[['Item_Identifier','Outlet_Identifier']])
# test_1_o = test_1_o.drop(['Item_Weight','Regular','Item_Type'],axis=1)
prediction61['Item_Outlet_Sales'] = [i[0] for i in model6.predict(test_1_os)]
prediction61.to_csv('submission61.csv',index=False)

In [145]:
# loss: 1128188.0000 
# - mean_squared_error: 1128188.0000 
# - val_loss: 1173223.1250 
# - val_mean_squared_error: 1173223.1250

# loss: 1127654.1250 - 
# mean_squared_error: 1127654.1250 - 
# val_loss: 1170178.6250 - 
# val_mean_squared_error: 1170178.6250



# model6.

In [146]:
# test_1_o = test_1_o.drop(['Item_Weight','Regular','Item_Type'],axis=1)

In [147]:
# test_1_o.shape

In [107]:
# SS2 = StandardScaler()

In [132]:
# test_1_os = SS2.fit_transform(test_1_o)

In [112]:
test_1_os.shape

(5681, 7)